In [1]:
// general includes
#include <iostream>   // std::cout|endl 
#include <memory>   // std::unique_ptr|shared_ptr
#include <cstdio>  // std::fopen|fprintf|fclose
struct Widget {
    int m;
};

## Quick Recap

...

## Goals for today

...

## Smart Pointers: `std::shared_ptr`

If a resource is intended to be shared (i.e., multiple handles exist simultaneously and are shared between participating entities) it is not straightforward to decide when to perform the final release of the resource: is is desired that this only happens after all participating entities are no more able to access the resource.
The `std::shared_ptr`[(cppref)](https://en.cppreference.com/w/cpp/memory/shared_ptr) solves this problem using *reference counting*: the number of valid handles (references) to a resource are tracked using a single counter variable per resource: 
- on construction of the first (original) handle the counter is set to '1'
- whenever an additional handle is constructed the counter is incremented
- whenever a handle is invalidated the counter is decremented
- if the counter is decremented to '0' the resource is released (as no more handles are present)

How to obtain a `shared_ptr`? Let's look at some examples:

In [2]:
std::shared_ptr<Widget> sp1 = std::make_shared<Widget>(); // using make_shared
std::shared_ptr<Widget> sp2 = std::shared_ptr<Widget>(new Widget{}); // ctor + new
std::unique_ptr<Widget> up1 = std::make_unique<Widget>(); // unique_ptr
std::shared_ptr<Widget> sp3 = std::move(up1); // from rvalue unique_ptr

// state of up1 here? 
// what is the reference count of sp1, sp2, sp3 here? 

**Is it possible to convert between `std::unique_ptr` and `std::shared_ptr`?**
- yes, an implicit conversion is available from `unique_ptr` to `shared_ptr` (from rvalues only).
- a conversion from `shared_ptr` to `unique_ptr` is not allowed (would need to invalidate all other references).

**Which restrictions (SMFs) can we expect to be lifted for `std::shared_ptr` (compared to `std::unique_ptr`)? What about pointer arithmetic and other operators?**
- copy assignment and copy construction is expected to be available
- pointer arithmetic and other operators are expected to be very similar

**Does the principle of the automatic release mechanism differ from `std::unique_ptr`?**
- identical in general; but the condition for its execution differs: only a decrement of the reference count to '0' triggers a release of the resource;
- this might also happen during a copy assignment (the refcount for the assigned resource is decreased), not only when a variable goes out of scope.

Reference counting example:

In [3]:
auto sp1 = std::make_shared<Widget>(); // (1) 1st sptr object
auto sp2 = std::make_shared<Widget>(); // (2) 2nd sptr object
auto sp3 = sp1; // (3) third sp object
sp2 = sp1; // (4) 

@0x7fe743b5a078

### Overhead: what does a `std::shared_ptr` look like
If we decide to use a `std::shared_ptr` instead of a `std::unique_ptr` or a raw pointer, what can we expect in terms of performance? To argue about this, let's look again at what (a small part of) a simplified implementation looks like:

In [4]:
#include <memory>

template <class T> class shared_ptr {
  struct ControlBlock {
    int count = 1;
    // some more members in stdlib
  };
  ControlBlock *cb; // (1) member A -> for refcounting
  T *ptr;           // (2) member B -> same as for unqiue_ptr

  void increment() {
    if (cb)
      ++cb->count;
  }
  void decrement() {
    if (cb && --cb->count == 0) {
      delete ptr; // wrapped resource
      delete cb;  // overhead
    }
  }

public:
  // usage: auto sp = shared_ptr<Widget>(new Widget{});
  shared_ptr(T *ptr) : cb(new ControlBlock()), ptr(ptr) {}
  shared_ptr(const shared_ptr &other) : cb(other.cb), ptr(other.ptr) {
    // (1) ???
    // 1) setup sharing of refcount and resource
    // 2) increment
  }
  shared_ptr(shared_ptr &&other) : cb(other.cb), ptr(other.ptr) {
    // (2) ???
    // 1) no inc/dec
    // 2) but: setup sharing of refcount and resource + set 'other' to
    // "NULL"
  }
  shared_ptr &operator=(const shared_ptr &other) {
    // (3) ???
    // 1) release managing the current resource (dec) + check if delete
    // 2) take over members from 'other' + inc
    return *this;
  }
  shared_ptr &operator=(shared_ptr &&other) {
    // (4) ???
    // 1) release managing the current resource (dec) + check if delete
    // 2) take over members from 'other' + set other to "NULL"
  }
  ~shared_ptr() {
    // (5) ???
    // 1) check if we are already the last owner (unique owner): if yes
    // free resource
    // 2) if not: decrement
  }
  T *operator->() const { return ptr; }
  T &operator*() const { return *ptr; }
};

template <typename T, typename... ARGS>
shared_ptr<T> make_shared(ARGS &&... args) {
  // do sth. here: -> use a single allocation for cb and ptr
  return shared_ptr<T>(new T(std::forward<ARGS>(args)...));
}

**Why can `make_shared` be an advantage compared to the 'regular' ctors?**
- reduce number of individual allocations

**What is expected to happen in the body of the SMFs marked above (to implement the reference counting)?**

- see above

It is visible that the construction (of the "original" handle ) triggers a subsequent dynamic allocation for the `ControlBlock`. The obtained pointer is stored as member `cb` in the `shared_pointer` additionally to the pointer of the managed object `ptr`. It is apparent that many of the SMFs will need to access the `ControlBlock`.
This means a `shared_ptr` does introduce some overhead (memory and access). This has to be considered for practical applications: if very small objects are managed by a `shared_ptr`, the relative increase of the memory footprint is not negligible. It might still be practical to use a `shared_ptr` for small objects, depending on the application context.


**Why is the `ControlBlock` a structure and not simply a integral type?**
- in this simple case, could be just a pointer to an (atomic) integer; in the stdlib, some more members are present in the `ControlBlock` structure to support advanced features of a `shared_ptr`: weak reference counting, and aliasing construction (access with offset).

**Would a static member be sufficient for reference counting, too?**
- no, a static member could only count references for all handles to shared pointers per type, but not per resource.

As we can see from the snippet above `shared_ptr`s can be copied and moved.
Let's examine some examples using again a `Widget` and a function returning a `share_ptr<Widget>`:

In [5]:
auto get_widget() {
  return std::make_shared<Widget>(); // using public constructors indirectly
}

In [6]:
auto sp = get_widget();                    // (1) one sp
std::cout << sp.use_count() << std::endl;  //  
auto sp2 = sp;                             // 
std::cout << sp.use_count() << std::endl;  //  
std::cout << sp2.use_count() << std::endl; // 
{
  auto sp3 = sp;                            // 
  std::cout << sp.use_count() << std::endl; // 
}
std::cout << sp.use_count() << std::endl; //  

1
2
2
3
2


@0x7fe73cec1de0

**What are the reference counts in the above snippet?**
- see above

**Capturing the managed object outside**


In [8]:
  Widget *ptr = nullptr;
  {
    auto sp = get_widget(); // (1a) obtain sp to a resource
    ptr = sp.get(); // (2) extract raw pointer (which is also a handle to the resource)
  } // (1b)
  ptr->m = 5; // (3) is this ok?

5

**Is it safe to perform the last line of the snippet above?**
- no, the resource `ptr` is pointing to is deallocated at (1b)

**Managing the same resource more than once**

In [10]:
  //...
  auto sp1 = get_widget();
  auto sp2 = get_widget();
  std::shared_ptr<Widget> sp3(sp1); // (1) copy ctor
  std::shared_ptr<Widget> sp4(sp2.get()); // (2) ctor from pointer

**Can you spot a Problem in the above snippet?**
- two independent reference counting control blocks exist for a single shared resource; this situation should never be created

**Providing a `shared_ptr` from "inside" #1**

In [11]:
struct Widget2 {
  int m;
  auto mfunc() { 
      return std::shared_ptr<Widget2>(this); // (1) is this ok?
  }
};

In [12]:
auto get_widget2() {
  return std::make_shared<Widget2>(); 
}

In [13]:
//...
auto sp = get_widget2();
auto sp2 = sp->mfunc(); // (2) is this ok?

**Is there a way to return a `shared_ptr` from within a managed class?**
- returning a `shared_ptr` constructed from `this` leads again to a "double management" (like above)
- to support such a situation, the object has to know that it is managed by a shared pointer and somehow needs access to the respective control block (see next example).

**Providing a `shared_ptr` from "inside" #2**

To overcome the problem in the previous snippet a solution is to inherit from `std::enable_shared_from_this`[(cppref)](https://en.cppreference.com/w/cpp/memory/enable_shared_from_this):


In [14]:
struct Widget3 : std::enable_shared_from_this<Widget3> {
  int m;
  auto mfunc() { 
      return shared_from_this(); // is this ok now? 
  }
};

In [15]:
auto get_widget3() { return std::make_shared<Widget3>(); }

In [17]:
auto sp = get_widget3();
auto sp2 = sp->mfunc(); // is this ok now? 

This adds a member of type `std::weak_ptr<Widget3>` to the structure which is connected to the managing `shared_ptr` instance on construction of the `shared_ptr` .

### std::weak_ptr 

A `std::weak_ptr`[(cppref)](https://en.cppreference.com/w/cpp/memory/weak_ptr) can be imagined to be a non-owning *potential* `shared_ptr` associated with a managing `shared_ptr`: it does not participate in the reference counting (actually it has its own reference counter for weak references). A `weak_ptr` does not influence the destruction of an object managed by a `shared_ptr` once the reference count goes to zero.
Nevertheless: a `weak_ptr` can be converted to a `shared_ptr`:

In [19]:
std::weak_ptr<Widget> wp;
{
  auto sp = std::make_shared<Widget>();
  wp = sp;
  auto sp2 = wp.lock();
}
if(!wp.expired()){
   auto sp = wp.lock();
   sp->m = 5;
}

This makes a `weak_ptr` suitable to be used in conjunction with `std::enable_shared_from_this`: 
it can provide a `shared_ptr` to the object via `shared_from_this()`.

### Thread safety

**Is a `std::shared_ptr` "thread-safe"?**
- "thread-safe" referes to increments and decrements (i.e. control block access)
- you can safely use the SMFs from different threads without further synchronization
- increments and decrements are typically synchronized using atomic operations
- about access to the managed resource: the share_pointer is **NOT** concerned what the managed resource provides in terms of having a thread-safe interface or not.

## Summary `std::shared_ptr`

- ...


### Discussion (from WS21):  

**How to express that a function argument** 

- points to (or references) a valid (i.e. non null) resource
- which (the resource) is managed by a smart pointer on the caller's site
- but the function will not pariticipate in ownership/lifecycle management

```cpp
void func(/*K ??? */ widget); // which type to choose?
// usage (call site)
shared_ptr<Widget> sp = ...;
func(/*K ??? */);
```
**Idea 1**: construct a new smart pointer with an empty deleter? 

```cpp
void func(shared_ptr<Widget> widget);
// usage
shared_ptr<Widget> sp = ...;
auto deleter = [](Widget *){};
func(shared_ptr<Widget,decltype(deleter)>(widget.get(),deleter))
```
**Idea 2**: use raw pointer? 

```cpp
void func(Widget* widget);
// usage
shared_ptr<Widget> sp = ...;
func(widget.get());
```

**Idea 3**: use a reference? 

```cpp
void func(Widget& widget);
// usage
shared_ptr<Widget> sp = ...;
func(*widget);
```
**Idea 4**: use std::weak_ptr?

```cpp
void func(weak_ptr<Widget> widget);
// usage
shared_ptr<Widget> sp = ...;
func(sp);
```

```cpp
#include <memory>

template <class T> class shared_ptr {
  struct ControlBlock {
    int count = 1;
    // some more members in stdlib
  };
  ControlBlock *cb; // (1) member A -> for refcounting
  T *ptr;           // (2) member B -> same as for unqiue_ptr

  void increment() {
    if (cb)
      ++cb->count;
  }
  void decrement() {
    if (cb && --cb->count == 0) {
      delete ptr; // wrapped resource
      delete cb;  // overhead
    }
  }

public:
  // usage: auto sp = shared_ptr<Widget>(new Widget{});
  shared_ptr(T *ptr) : cb(new ControlBlock()), ptr(ptr) {}
  shared_ptr(const shared_ptr &other) : cb(other.cb), ptr(other.ptr) {
    // (1) ???
    // 1) setup sharing of refcount and resource
    // 2) increment
  }
  shared_ptr(shared_ptr &&other) : cb(other.cb), ptr(other.ptr) {
    // (2) ???
    // 1) no inc/dec
    // 2) but: setup sharing of refcount and resource + set 'other' to
    // "NULL"
  }
  shared_ptr &operator=(const shared_ptr &other) {
    // (3) ???
    // 1) release managing the current resource (dec) + check if delete
    // 2) take over members from 'other' + inc
    return *this;
  }
  shared_ptr &operator=(shared_ptr &&other) {
    // (4) ???
    // 1) release managing the current resource (dec) + check if delete
    // 2) take over members from 'other' + set other to "NULL"
  }
  ~shared_ptr() {
    // (5) ???
    // 1) check if we are already the last owner (unique owner): if yes
    // free resource
    // 2) if not: decrement
  }
  T *operator->() const { return ptr; }
  T &operator*() const { return *ptr; }
};

template <typename T, typename... ARGS>
shared_ptr<T> make_shared(ARGS &&... args) {
  // do sth. here: -> use a single allocation for cb and ptr
  return shared_ptr<T>(new T(std::forward<ARGS>(args)...));
}


```